## Working with Different Types of Data


### Step 1: Initialize PySpark Session


In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("day3").getOrCreate()


23/08/21 16:29:20 WARN Utils: Your hostname, FM-PC-LT-321 resolves to a loopback address: 127.0.1.1; using 192.168.101.8 instead (on interface wlp0s20f3)
23/08/21 16:29:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/21 16:29:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/21 16:29:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Step 2: Load the Dataset


In [36]:
# Load the Chipotle dataset into a Spark DataFrame
data_path = "../data/titanic.csv"  # Replace with the actual path
titanic_df = spark.read.csv(data_path, header=True, inferSchema=True)

# Load the Chipotle dataset into a Spark DataFrame
data_path = '../data/chipotle.csv' # Replace with the actual path
chipotle_df = spark.read.csv(data_path, header=True, inferSchema=True)

# Load the Chipotle dataset into a Spark DataFrame
data_path = '../data/kalimati_tarkari_dataset.csv' # Replace with the actual path
kalimati_df = spark.read.csv(data_path, header=True, inferSchema=True)


In [9]:
print(titanic.printSchema(),chipotle.printSchema(),kalimati_tarkari_dataset.printSchema())

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

root
 |-- _c0: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)

root
 |-- SN: integer (nullable = true)
 |-- Commodity: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Minimum: double (nullable = true)
 |-- Maximum: double (nullable = true)
 |-- Average: double (nullable = true)

None None N

### Converting to Spark Types:

Question: Load the "titanic" dataset and convert the "Fare" column from double to integer.




root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: integer (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



### Working with Booleans:

Question: Load the "titanic" dataset and add a new column "IsAdult" that indicates whether a passenger is an adult (age >= 18) or not.

+-----------+--------------------+----+-------+
|PassengerId|                Name| Age|IsAdult|
+-----------+--------------------+----+-------+
|          1|Braund, Mr. Owen ...|22.0|   true|
|          2|Cumings, Mrs. Joh...|38.0|   true|
|          3|Heikkinen, Miss. ...|26.0|   true|
|          4|Futrelle, Mrs. Ja...|35.0|   true|
|          5|Allen, Mr. Willia...|35.0|   true|
|          6|    Moran, Mr. James|null|  false|
|          7|McCarthy, Mr. Tim...|54.0|   true|
|          8|Palsson, Master. ...| 2.0|  false|
|          9|Johnson, Mrs. Osc...|27.0|   true|
|         10|Nasser, Mrs. Nich...|14.0|  false|
|         11|Sandstrom, Miss. ...| 4.0|  false|
|         12|Bonnell, Miss. El...|58.0|   true|
|         13|Saundercock, Mr. ...|20.0|   true|
|         14|Andersson, Mr. An...|39.0|   true|
|         15|Vestrom, Miss. Hu...|14.0|  false|
|         16|Hewlett, Mrs. (Ma...|55.0|   true|
|         17|Rice, Master. Eugene| 2.0|  false|
|         18|Williams, Mr. Cha...|null| 

### Working with Numbers:

Question: Load the "titanic" dataset and calculate the average age of male and female passengers separately.

+------+------------------+
|   Sex|            AvgAge|
+------+------------------+
|female|27.915708812260537|
|  male| 30.72664459161148|
+------+------------------+



### Working with Strings:

Question: Load the "chipotle" dataset and find the item names containing the word "Chicken."

+---+--------+--------+--------------------+--------------------+----------+
|_c0|order_id|quantity|           item_name|  choice_description|item_price|
+---+--------+--------+--------------------+--------------------+----------+
|  4|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|  5|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
| 11|       6|       1|Chicken Crispy Tacos|[Roasted Chili Co...|    $8.75 |
| 12|       6|       1|  Chicken Soft Tacos|[Roasted Chili Co...|    $8.75 |
| 13|       7|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $11.25 |
| 16|       8|       1|     Chicken Burrito|[Tomatillo-Green ...|    $8.49 |
| 17|       9|       1|     Chicken Burrito|[Fresh Tomato Sal...|    $8.49 |
| 19|      10|       1|        Chicken Bowl|[Tomatillo Red Ch...|    $8.75 |
| 23|      12|       1|     Chicken Burrito|[[Tomatillo-Green...|   $10.98 |
| 26|      13|       1|        Chicken Bowl|[Roasted Chili Co...|    $8.49 |

23/08/21 17:33:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , order_id, quantity, item_name, choice_description, item_price
 Schema: _c0, order_id, quantity, item_name, choice_description, item_price
Expected: _c0 but found: 
CSV file: file:///home/fm-pc-lt-321/Desktop/fuse/pyspark/assignments/trainee%20exercises/data/chipotle.csv


### Regular Expressions:

Question: Load the "chipotle" dataset and find the items with names that start with "Ch" followed by any character.



+---+--------+--------+--------------------+--------------------+----------+
|_c0|order_id|quantity|           item_name|  choice_description|item_price|
+---+--------+--------+--------------------+--------------------+----------+
|  0|       1|       1|Chips and Fresh T...|                null|    $2.39 |
|  3|       1|       1|Chips and Tomatil...|                null|    $2.39 |
|  4|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|  5|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
| 10|       5|       1| Chips and Guacamole|                null|    $4.45 |
| 11|       6|       1|Chicken Crispy Tacos|[Roasted Chili Co...|    $8.75 |
| 12|       6|       1|  Chicken Soft Tacos|[Roasted Chili Co...|    $8.75 |
| 13|       7|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $11.25 |
| 14|       7|       1| Chips and Guacamole|                null|    $4.45 |
| 15|       8|       1|Chips and Tomatil...|                null|    $2.39 |

23/08/21 17:35:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , order_id, quantity, item_name, choice_description, item_price
 Schema: _c0, order_id, quantity, item_name, choice_description, item_price
Expected: _c0 but found: 
CSV file: file:///home/fm-pc-lt-321/Desktop/fuse/pyspark/assignments/trainee%20exercises/data/chipotle.csv


### Working with Nulls in Data:

Question: Load the "titanic" dataset and count the number of passengers with missing age information.



Number of passengers with missing age: 177


### Coalesce
Question: Utilizing the Chipotle dataset, use the coalesce function to combine the "item_name" and "choice_description" columns into a new column named "OrderDetails." Display the first 5 rows of the resulting DataFrame.

+---+--------+--------+--------------------+--------------------+----------+--------------------+
|_c0|order_id|quantity|           item_name|  choice_description|item_price|        OrderDetails|
+---+--------+--------+--------------------+--------------------+----------+--------------------+
|  0|       1|       1|Chips and Fresh T...|                null|    $2.39 |Chips and Fresh T...|
|  1|       1|       1|                Izze|        [Clementine]|    $3.39 |                Izze|
|  2|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |    Nantucket Nectar|
|  3|       1|       1|Chips and Tomatil...|                null|    $2.39 |Chips and Tomatil...|
|  4|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |        Chicken Bowl|
+---+--------+--------+--------------------+--------------------+----------+--------------------+
only showing top 5 rows



23/08/21 17:51:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , order_id, quantity, item_name, choice_description, item_price
 Schema: _c0, order_id, quantity, item_name, choice_description, item_price
Expected: _c0 but found: 
CSV file: file:///home/fm-pc-lt-321/Desktop/fuse/pyspark/assignments/trainee%20exercises/data/chipotle.csv


### ifnull, nullIf, nvl, and nvl2

Question: Replace the null values in the "Age" column of the Titanic dataset with the average age.

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|             35.0|    0|    0|          373450|

### drop

Question: Remove the "Cabin" column from the Titanic dataset.


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877| 8.4583|       Q|
|          7|      

### fill

Question: Fill the null values in the "Age" column of the Titanic dataset with a default age of 30.

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|30.0|    0|    0|      

###  replace

Question: Replace the gender "male" with "M" and "female" with "F" in the "Sex" column of the Titanic dataset.

+-----------+--------+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  M|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|  F|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|  F|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|  F|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  M|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  M|null|    0|    0|          330877| 8.4583| null|  

### 6. Working with Complex Types: Structs

Question: Create a new DataFrame from the Kalimati Tarkari dataset, including a new column "PriceRange" that is a struct containing "Minimum" and "Maximum" prices for each commodity.

+---+--------------------+----------+----+-------+-------+-------+------------+
|SN |Commodity           |Date      |Unit|Minimum|Maximum|Average|PriceRange  |
+---+--------------------+----------+----+-------+-------+-------+------------+
|0  |Tomato Big(Nepali)  |2013-06-16|Kg  |35.0   |40.0   |37.5   |{35.0, 40.0}|
|1  |Tomato Small(Local) |2013-06-16|Kg  |26.0   |32.0   |29.0   |{26.0, 32.0}|
|2  |Potato Red          |2013-06-16|Kg  |20.0   |21.0   |20.5   |{20.0, 21.0}|
|3  |Potato White        |2013-06-16|Kg  |15.0   |16.0   |15.5   |{15.0, 16.0}|
|4  |Onion Dry (Indian)  |2013-06-16|Kg  |28.0   |30.0   |29.0   |{28.0, 30.0}|
|5  |Carrot(Local)       |2013-06-16|Kg  |30.0   |35.0   |32.5   |{30.0, 35.0}|
|6  |Cabbage(Local)      |2013-06-16|Kg  |6.0    |10.0   |8.0    |{6.0, 10.0} |
|7  |Cauli Local         |2013-06-16|Kg  |30.0   |35.0   |32.5   |{30.0, 35.0}|
|8  |Raddish Red         |2013-06-16|Kg  |35.0   |40.0   |37.5   |{35.0, 40.0}|
|9  |Raddish White(Local)|2013-06-16|Kg 

### Working with Complex Types: Arrays
Question: Create a new DataFrame from the Kalimati Tarkari dataset, including a new column "CommodityList" that is an array of all the commodities.


+---+--------------------+----------+----+-------+-------+-------+----------------------+
|SN |Commodity           |Date      |Unit|Minimum|Maximum|Average|CommodityList         |
+---+--------------------+----------+----+-------+-------+-------+----------------------+
|0  |Tomato Big(Nepali)  |2013-06-16|Kg  |35.0   |40.0   |37.5   |[Tomato Big(Nepali)]  |
|1  |Tomato Small(Local) |2013-06-16|Kg  |26.0   |32.0   |29.0   |[Tomato Small(Local)] |
|2  |Potato Red          |2013-06-16|Kg  |20.0   |21.0   |20.5   |[Potato Red]          |
|3  |Potato White        |2013-06-16|Kg  |15.0   |16.0   |15.5   |[Potato White]        |
|4  |Onion Dry (Indian)  |2013-06-16|Kg  |28.0   |30.0   |29.0   |[Onion Dry (Indian)]  |
|5  |Carrot(Local)       |2013-06-16|Kg  |30.0   |35.0   |32.5   |[Carrot(Local)]       |
|6  |Cabbage(Local)      |2013-06-16|Kg  |6.0    |10.0   |8.0    |[Cabbage(Local)]      |
|7  |Cauli Local         |2013-06-16|Kg  |30.0   |35.0   |32.5   |[Cauli Local]         |
|8  |Raddi

### Working with Complex Types: explode

Question: Explode the "CommodityList" array column from the previous step to generate a new row for each commodity in the list.

+---+----------+----+-------+-------+-------+--------------------+
|SN |Date      |Unit|Minimum|Maximum|Average|Commodity           |
+---+----------+----+-------+-------+-------+--------------------+
|0  |2013-06-16|Kg  |35.0   |40.0   |37.5   |Tomato Big(Nepali)  |
|1  |2013-06-16|Kg  |26.0   |32.0   |29.0   |Tomato Small(Local) |
|2  |2013-06-16|Kg  |20.0   |21.0   |20.5   |Potato Red          |
|3  |2013-06-16|Kg  |15.0   |16.0   |15.5   |Potato White        |
|4  |2013-06-16|Kg  |28.0   |30.0   |29.0   |Onion Dry (Indian)  |
|5  |2013-06-16|Kg  |30.0   |35.0   |32.5   |Carrot(Local)       |
|6  |2013-06-16|Kg  |6.0    |10.0   |8.0    |Cabbage(Local)      |
|7  |2013-06-16|Kg  |30.0   |35.0   |32.5   |Cauli Local         |
|8  |2013-06-16|Kg  |35.0   |40.0   |37.5   |Raddish Red         |
|9  |2013-06-16|Kg  |25.0   |30.0   |27.5   |Raddish White(Local)|
|10 |2013-06-16|Kg  |16.0   |18.0   |17.0   |Brinjal Long        |
|11 |2013-06-16|Kg  |20.0   |22.0   |21.0   |Brinjal Round    

### Working with Complex Types: Maps

Question: Create a new DataFrame from the Kalimati Tarkari dataset, including a new column "PriceMap" that is a map with "Commodity" as the key and "Average" price as the value.
Answer:

+---+--------------------+----------+----+-------+-------+-------+------------------------------+
|SN |Commodity           |Date      |Unit|Minimum|Maximum|Average|PriceMap                      |
+---+--------------------+----------+----+-------+-------+-------+------------------------------+
|0  |Tomato Big(Nepali)  |2013-06-16|Kg  |35.0   |40.0   |37.5   |{Tomato Big(Nepali) -> 37.5}  |
|1  |Tomato Small(Local) |2013-06-16|Kg  |26.0   |32.0   |29.0   |{Tomato Small(Local) -> 29.0} |
|2  |Potato Red          |2013-06-16|Kg  |20.0   |21.0   |20.5   |{Potato Red -> 20.5}          |
|3  |Potato White        |2013-06-16|Kg  |15.0   |16.0   |15.5   |{Potato White -> 15.5}        |
|4  |Onion Dry (Indian)  |2013-06-16|Kg  |28.0   |30.0   |29.0   |{Onion Dry (Indian) -> 29.0}  |
|5  |Carrot(Local)       |2013-06-16|Kg  |30.0   |35.0   |32.5   |{Carrot(Local) -> 32.5}       |
|6  |Cabbage(Local)      |2013-06-16|Kg  |6.0    |10.0   |8.0    |{Cabbage(Local) -> 8.0}       |
|7  |Cauli Local    

### Working with JSON

Question: Convert the "kalimati_df" DataFrame to JSON format and write it to a JSON file.